In [12]:
import os
os.chdir("..")
from pathlib import Path
import cv2
import torch
import numpy as np
from tqdm import tqdm
from torchvision import transforms
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

In [ ]:
# VIDEO_DIR = Path("data/videos")
# FRAME_OUTPUT_DIR = Path("data/frames")
# NUM_FRAMES = 20
# RESIZE_DIMS = (128, 128)

# def extract_frames_from_video(video_path, num_frames, resize_dims):
#     cap = cv2.VideoCapture(str(video_path))
#     total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

#     if total_frames < num_frames:
#         raise ValueError(f"Video {video_path} has fewer than {num_frames} frames.")

#     frame_indices = [int(i * total_frames / num_frames) for i in range(num_frames)]
#     frames = []
#     current_idx = 0
#     read_idx = 0

#     while cap.isOpened():
#         ret, frame = cap.read()
#         if not ret:
#             break

#         if read_idx == frame_indices[current_idx]:
#             resized = cv2.resize(frame, resize_dims)
#             frames.append(resized)
#             current_idx += 1
#             if current_idx >= len(frame_indices):
#                 break

#         read_idx += 1

#     cap.release()
#     return frames

# # === Frame Extraction ===
# gesture_classes = sorted([f.name for f in VIDEO_DIR.iterdir() if f.is_dir()])

# for gesture in gesture_classes:
#     (FRAME_OUTPUT_DIR / gesture).mkdir(parents=True, exist_ok=True)

#     gesture_video_dir = VIDEO_DIR / gesture
#     gesture_output_dir = FRAME_OUTPUT_DIR / gesture

#     for video_file in os.listdir(gesture_video_dir):
#         if not video_file.endswith(".mp4"):
#             continue

#         video_path = gesture_video_dir / video_file
#         output_subdir = gesture_output_dir / video_file.replace(".mp4", "")
#         output_subdir.mkdir(parents=True, exist_ok=True)

#         print(f"Processing video: {video_file}...")

#         try:
#             frames = extract_frames_from_video(video_path, NUM_FRAMES, RESIZE_DIMS)
#             for idx, frame in enumerate(frames):
#                 frame_path = output_subdir / f"{idx:03d}.jpg"
#                 cv2.imwrite(str(frame_path), frame)
#         except Exception as e:
#             print(f"Failed to process {video_file}: {e}")

# print("Frame extraction completed.")

print(os.listdir("data"))  # should print: ['videos']


c:\Users\Dharan\OneDrive\Documents\SkillCraft Tasks\Task 4\SCT_ML_4\Notebook
